In [163]:
from openai import OpenAI
import pathlib
import textwrap
import os
import time
from jinja2 import Environment, FileSystemLoader
from tqdm.notebook import tqdm
from IPython.display import display
import re
from IPython.display import Markdown
import json

In [164]:
def replace_math_delimiters(content):
    pattern = r'\$(.*?)\$'
    new_content = re.sub(pattern, r'\\(\1\\)', content)
    return new_content

In [168]:
def create_def_files(questions, chapter, set_name):
    s = """assignmentType      = default
openDate          = 08/12/2024 at 11:59pm EDT
dueDate           = 08/19/2024 at 11:59pm EDT
answerDate        = 08/21/2024 at 11:59pm EDT
problemListV2\n\n"""
    problem = ""
    i = 1;
    for section in questions[chapter]:
        section_name = ('_'.join(section.split('-')[2:]))

        problem = problem + f"""problem_start
problem_id = {i}
source_file = {set_name}/{section_name}.pg
value = 1
problem_end\n\n"""
        i+=1
    return(s + problem)

In [169]:
with open("./generated_files/MultiOpenStax_Definitions20240812_0452.json", "r") as f:
    questions = json.load(f)
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('./generated_files/WebWork/truefalse_template.pg.j2')

In [170]:
for chapter in questions:
    set_name = "set" + ''.join(chapter.split())
    relative_location = f"generated_files/WebWork/MultiVariableCalculus/{set_name}"
    def_file = create_def_files(questions, chapter, set_name)

    if (not os.path.exists(relative_location)):
        os.mkdir(relative_location)
    with open(relative_location + ".def", "w") as f:
        f.write(def_file)

    
    for section in questions[chapter]:
        section_name = ('_'.join(section.split('-')[2:]))
        questions_data = (questions[chapter][section])
        data = {
            "questions": questions_data,
            "num" : len(questions_data),
        }
        output = template.render(data, custom_function=replace_math_delimiters)
        output_filename = relative_location + f"/{section_name}.pg"
        with open(output_filename, 'w') as f:
            f.write(output)